# Data Collection for Europe Base Port Container Price Prediction

This notebook focuses on gathering all the raw data needed for our 1-week ahead container price forecasting project. We will collect data from three different sources and save it for later processing.

## What is data collection?

Data collection is the first step in any data science project. It involves gathering raw information from various sources like files, databases, or APIs (Application Programming Interfaces, which are ways for programs to talk to each other over the internet). Think of it like gathering ingredients before cooking a meal.

## What are we predicting?

**Target**: Europe Base Port container prices (1 week ahead)

**Base Ports Definition**: Average shipping cost for a 40-foot container from Shanghai/China to major European ports including Rotterdam (Netherlands), Hamburg (Germany), London (UK), and Antwerp (Belgium).

**Why these ports**: These represent the main entry points for Asian goods into Europe and provide a standard benchmark for European route pricing.

## Our data sources:

1. Shanghai Containerized Freight Index (local CSV file) - Main price data
2. Oil prices (from Yahoo Finance API Hopefully) - Cost factor affecting shipping / (Not working looking for work around)
3. Geopolitical disruption data (from GDELT via BigQuery) - Black swan event indicators

**Note**: We use GDELT data exported from Google BigQuery for historical coverage (2018-2025). This provides weekly disruption metrics including conflict events, severe incidents, and sentiment analysis for shipping-critical regions.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import time
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## Step 1: Load the Shanghai Containerized Freight Index

This is our main dataset. It contains weekly freight prices for shipping containers from Shanghai to various destinations around the world. We are specifically interested in the Europe Base Port price, which represents the average cost to ship a container from Shanghai/China to major European ports (Rotterdam, Hamburg, London, Antwerp).

In [2]:
try:
    # Load the raw CSV file
    # header=1 means the actual column names are in the second row (row 1, counting from 0)
    raw_df = pd.read_csv('data/Shanghai_Containerized_Freight_Index.csv', header=1)
    print(f"Successfully loaded the CSV file with {raw_df.shape[0]} rows and {raw_df.shape[1]} columns")
    
    # Display the first few rows to see what the data looks like
    print("\nFirst 5 rows of the raw data:")
    print(raw_df.head())
    
except FileNotFoundError:
    print("Error: The file 'data/Shanghai_Containerized_Freight_Index.csv' was not found.")
    print("Please make sure it is in the data folder.")

Successfully loaded the CSV file with 385 rows and 5 columns

First 5 rows of the raw data:
  the period (YYYY-MM-DD)  Comprehensive Index  Europe (Base port)  \
0                1/5/2018               816.58                 888   
1               1/12/2018               839.72                 897   
2               1/19/2018               840.36                 891   
3               1/26/2018               858.60                 907   
4                2/2/2018               883.59                 912   

   Mediterranean (Base port)  Persian Gulf and Red Sea (Dubai)  
0                        738                               433  
1                        759                               450  
2                        761                               572  
3                        772                               631  
4                        797                               611  


## Step 2: Select and rename the columns we need

The dataset has many columns for different routes, but we only need a few for our Europe Base Port prediction. We will select the date column, the overall freight index (SCFI), and the Europe Base Port price.

### Why select only these columns?

Europe Base Port: Our prediction target - average cost from Shanghai to Rotterdam/Hamburg/London/Antwerp

SCFI_Index: Overall freight market indicator (note: highly correlated with Europe prices, so we may exclude it from final models)

Date: Needed for time series analysis and chronological ordering

In [3]:
# Select only the columns we need
df_freight = raw_df[['the period (YYYY-MM-DD)', 'Comprehensive Index', 'Europe (Base port)']].copy()

# Rename columns to simpler names
df_freight.columns = ['Date', 'SCFI_Index', 'Europe_Base_Price']

print("Selected and renamed columns:")
print(df_freight.columns.tolist())
print(f"\nDataset now has {df_freight.shape[0]} rows and {df_freight.shape[1]} columns")
print("\nFirst 5 rows:")
print(df_freight.head())

Selected and renamed columns:
['Date', 'SCFI_Index', 'Europe_Base_Price']

Dataset now has 385 rows and 3 columns

First 5 rows:
        Date  SCFI_Index  Europe_Base_Price
0   1/5/2018      816.58                888
1  1/12/2018      839.72                897
2  1/19/2018      840.36                891
3  1/26/2018      858.60                907
4   2/2/2018      883.59                912


## Step 3: Convert date strings to proper date format

Right now, the Date column is just text. We need to convert it to a proper datetime format so Python understands it represents actual dates.

### What is datetime?

datetime is a special data type in Python that represents dates and times. It allows us to do things like sort by date, calculate time differences, and extract parts of a date (like the month or year).

In [4]:
df_freight['Date'] = pd.to_datetime(df_freight['Date'], format='%m/%d/%Y', errors='coerce')

# Check how many dates were successfully converted
valid_dates = df_freight['Date'].notna().sum()
total_rows = len(df_freight)
print(f"Successfully converted {valid_dates} out of {total_rows} dates")

# Remove rows where date conversion failed
df_freight.dropna(subset=['Date'], inplace=True)
print(f"After removing invalid dates: {len(df_freight)} rows remaining")

# Set the Date column as the index (the row identifier)
df_freight.set_index('Date', inplace=True)

print("\nDate conversion complete!")
print(df_freight.head())

Successfully converted 385 out of 385 dates
After removing invalid dates: 385 rows remaining

Date conversion complete!
            SCFI_Index  Europe_Base_Price
Date                                     
2018-01-05      816.58                888
2018-01-12      839.72                897
2018-01-19      840.36                891
2018-01-26      858.60                907
2018-02-02      883.59                912


## Step 4: Convert price columns to numbers

Sometimes data is read as text even when it represents numbers. We need to ensure our price columns are in numeric format so we can do calculations with them.

### What is numeric conversion?

This process takes text that looks like numbers (like "123.45") and converts it to actual numbers that Python can use for math operations.

In [5]:
# Convert price columns to numeric format
for col in ['SCFI_Index', 'Europe_Base_Price']:
    df_freight[col] = pd.to_numeric(df_freight[col], errors='coerce')
    print(f"Converted {col} to numeric type")

# Remove any rows with missing values
# This ensures we have complete data for all rows
before_drop = len(df_freight)
df_freight.dropna(inplace=True)
after_drop = len(df_freight)

print(f"\nRemoved {before_drop - after_drop} rows with missing values")
print(f"Final freight dataset: {after_drop} rows")
print(f"Date range: {df_freight.index.min().strftime('%Y-%m-%d')} to {df_freight.index.max().strftime('%Y-%m-%d')}")

print("\nFinal cleaned freight data:")
print(df_freight.head())

Converted SCFI_Index to numeric type
Converted Europe_Base_Price to numeric type

Removed 0 rows with missing values
Final freight dataset: 385 rows
Date range: 2018-01-05 to 2025-08-22

Final cleaned freight data:
            SCFI_Index  Europe_Base_Price
Date                                     
2018-01-05      816.58                888
2018-01-12      839.72                897
2018-01-19      840.36                891
2018-01-26      858.60                907
2018-02-02      883.59                912


## Step 5: Fetch oil price data

Oil prices affect shipping costs since ships use fuel. We will download historical oil prices using multiple fallback methods.

### Option 1: Yahoo Finance API (Primary)

We try to fetch data from Yahoo Finance using the `yfinance` library. This is free and usually reliable, though it can occasionally fail due to rate limiting or API changes.

**Tickers tried:**
- **USO**: United States Oil Fund ETF (most reliable)
- **CL=F**: WTI Crude Oil Futures
- **BZ=F**: Brent Crude Oil Futures  
- **XLE**: Energy Select Sector SPDR Fund (energy sector)

### Option 2: Manual CSV File (Backup)

If Yahoo Finance fails, you can provide your own oil price data:

1. **Download oil prices** from one of these sources:
   - **EIA (U.S. Energy Information Administration)**: https://www.eia.gov/dnav/pet/hist/RWTCD.htm
     - Free, official U.S. government data
     - Download as Excel/CSV and save as `oil_prices_manual.csv`
   - **FRED (Federal Reserve Economic Data)**: https://fred.stlouisfed.org/series/DCOILWTICO
     - Free historical WTI crude oil prices
     - Click "Download" → CSV format
   - **Quandl/Nasdaq Data Link**: https://data.nasdaq.com/
     - Free tier available with API key
     
2. **Format the CSV** with two columns:
   ```
   Date,Price
   2018-01-01,60.37
   2018-01-02,61.44
   ...
   ```
   
3. **Save as** `oil_prices_manual.csv` in the same directory as this notebook

### Option 3: Synthetic Data (Last Resort)

If no real data is available, we create synthetic oil prices for testing purposes only. This uses a random walk model with realistic price ranges ($50-$120) but should NOT be used for actual predictions.

### Why Brent/WTI Crude Oil?

Brent Crude and WTI (West Texas Intermediate) are major oil benchmarks used for global pricing. Their price movements indicate changes in shipping fuel costs.

In [6]:
print("=" * 70)
print("FETCHING OIL/ENERGY PRICE DATA")
print("=" * 70)

# Get the date range from our freight data
start_date = df_freight.index.min().strftime('%Y-%m-%d')
end_date = df_freight.index.max().strftime('%Y-%m-%d')

print(f"\nRequesting data from {start_date} to {end_date}")

df_oil = pd.DataFrame()

# Option 1: Try Yahoo Finance first (most reliable when it works)
print("\n--- Option 1: Trying Yahoo Finance API ---")

oil_tickers = [
    ('USO', 'United States Oil Fund ETF'),  # Oil ETF - most reliable
    ('CL=F', 'WTI Crude Oil Futures'),  # WTI Futures
    ('BZ=F', 'Brent Crude Oil Futures'),  # Brent Futures
    ('XLE', 'Energy Select Sector SPDR Fund'),  # Energy sector ETF
]

for ticker, name in oil_tickers:
    print(f"\nTrying {name} ({ticker})...")
    
    try:
        # Download the oil price data with a longer timeout
        temp_df = yf.download(ticker, start=start_date, end=end_date, progress=False, timeout=15)

        if not temp_df.empty and len(temp_df) > 10:  # Need at least some data
            # Sometimes yfinance returns data with multiple column levels, we need to flatten it
            if temp_df.columns.nlevels > 1:
                temp_df.columns = temp_df.columns.droplevel(1)
            
            # Keep only the closing price and rename it
            df_oil = temp_df[['Close']].rename(columns={'Close': 'Oil_Price'})
            
            print(f"Success: Successfully fetched {len(df_oil)} days of oil price data from {name}")
            print(f"  Date range: {df_oil.index.min().strftime('%Y-%m-%d')} to {df_oil.index.max().strftime('%Y-%m-%d')}")
            print(f"  Price range: ${df_oil['Oil_Price'].min():.2f} to ${df_oil['Oil_Price'].max():.2f}")
            print(f"  Average price: ${df_oil['Oil_Price'].mean():.2f}")
            print("\nFirst 5 rows of oil data:")
            print(df_oil.head())
            break  # Success! Exit the loop
        else:
            print(f"Error: No data or insufficient data returned for {ticker}")
            
    except Exception as e:
        print(f"Error: Error with {ticker}: {str(e)[:150]}")
        continue

# Option 2: Try loading from a manual CSV file if API failed
if df_oil.empty:
    print("\n--- Option 2: Trying manual CSV file ---")
    try:
        # Check if user has provided a manual oil price CSV
        oil_csv_path = 'oil_prices_manual.csv'
        df_oil = pd.read_csv(oil_csv_path, parse_dates=['Date'], index_col='Date')
        
        # Filter to our date range
        df_oil = df_oil[(df_oil.index >= start_date) & (df_oil.index <= end_date)]
        
        if not df_oil.empty:
            # Rename to standard column name
            if 'Price' in df_oil.columns:
                df_oil = df_oil[['Price']].rename(columns={'Price': 'Oil_Price'})
            elif 'Close' in df_oil.columns:
                df_oil = df_oil[['Close']].rename(columns={'Close': 'Oil_Price'})
            
            print(f"Success: Loaded {len(df_oil)} days from manual CSV: {oil_csv_path}")
            print(f"  Date range: {df_oil.index.min().strftime('%Y-%m-%d')} to {df_oil.index.max().strftime('%Y-%m-%d')}")
            print(f"  Price range: ${df_oil['Oil_Price'].min():.2f} to ${df_oil['Oil_Price'].max():.2f}")
        else:
            print(f"Error: CSV file found but no data in date range")
            
    except FileNotFoundError:
        print(f"Error: Manual CSV file not found: {oil_csv_path}")
    except Exception as e:
        print(f"Error: Error loading manual CSV: {str(e)[:100]}")

# Option 3: Create synthetic oil price data based on historical patterns (last resort)
if df_oil.empty:
    print("\n--- Option 3: Creating synthetic placeholder data ---")
    print("Warning: No real oil data available. Creating synthetic data for demonstration.")
    print("This should only be used for testing. For production, obtain real oil price data.")
    
    # Create date range matching freight data
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Create synthetic oil prices with realistic values and volatility
    # Base price around $70-80 with random walk
    np.random.seed(42)  # For reproducibility
    base_price = 75.0
    random_walk = np.random.randn(len(date_range)).cumsum() * 2  # Random walk with std=2
    synthetic_prices = base_price + random_walk
    
    # Clip to reasonable range (50-120)
    synthetic_prices = np.clip(synthetic_prices, 50, 120)
    
    df_oil = pd.DataFrame({
        'Oil_Price': synthetic_prices
    }, index=date_range)
    
    print(f"Success: Created {len(df_oil)} days of synthetic oil price data")
    print(f"  Date range: {df_oil.index.min().strftime('%Y-%m-%d')} to {df_oil.index.max().strftime('%Y-%m-%d')}")
    print(f"  Price range: ${df_oil['Oil_Price'].min():.2f} to ${df_oil['Oil_Price'].max():.2f}")
    print(f"  Average: ${df_oil['Oil_Price'].mean():.2f}")
    print("\nWarning: This is SYNTHETIC data. Replace with real data for actual predictions.")

print("\n" + "=" * 70)
print("OIL DATA COLLECTION COMPLETE")
print("=" * 70)
print(f"Total days: {len(df_oil)}")
print(f"Data source: {'Yahoo Finance' if not df_oil.empty and 'synthetic' not in str(df_oil.index.name) else 'Synthetic/Manual'}")
print("=" * 70)

FETCHING OIL/ENERGY PRICE DATA

Requesting data from 2018-01-05 to 2025-08-22

--- Option 1: Trying Yahoo Finance API ---

Trying United States Oil Fund ETF (USO)...


Failed to get ticker 'USO' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['USO']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')

1 Failed download:
['USO']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Error: No data or insufficient data returned for USO

Trying WTI Crude Oil Futures (CL=F)...


Failed to get ticker 'CL=F' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['CL=F']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')

1 Failed download:
['CL=F']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Error: No data or insufficient data returned for CL=F

Trying Brent Crude Oil Futures (BZ=F)...


Failed to get ticker 'BZ=F' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:
['BZ=F']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')

1 Failed download:
['BZ=F']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Error: No data or insufficient data returned for BZ=F

Trying Energy Select Sector SPDR Fund (XLE)...


Failed to get ticker 'XLE' reason: Expecting value: line 1 column 1 (char 0)

1 Failed download:

1 Failed download:
['XLE']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['XLE']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Error: No data or insufficient data returned for XLE

--- Option 2: Trying manual CSV file ---
Error: Manual CSV file not found: oil_prices_manual.csv

--- Option 3: Creating synthetic placeholder data ---
This should only be used for testing. For production, obtain real oil price data.
Success: Created 2787 days of synthetic oil price data
  Date range: 2018-01-05 to 2025-08-22
  Price range: $50.00 to $120.00
  Average: $103.58


OIL DATA COLLECTION COMPLETE
Total days: 2787
Data source: Yahoo Finance


## Step 6: Load the new "black swan" geopolitical disruption data

We now load the new, richer BigQuery dataset that contains specific geopolitical and black swan event metrics for shipping-critical regions.

In [7]:
print("=" * 70)
print("LOADING NEW BLACK SWAN GEOPOLITICAL DISRUPTION DATA")
print("=" * 70)

# Helper function to convert ISO year/week to Friday date
def iso_to_friday_date(row):
    """Converts ISO year and week to the Friday of that week."""
    return datetime.fromisocalendar(int(row['iso_year']), int(row['week']), 5)

# Load the new BigQuery export
try:
    news_file_path = 'data/bq-results-20251021-090045-1761037274833.csv'
    df_news_raw = pd.read_csv(news_file_path)
    print(f"Success: Loaded {len(df_news_raw)} weekly records from {news_file_path}")
    print(f"  Columns: {df_news_raw.columns.tolist()}")

    # Remove week 53 entries as they cause invalid week errors
    before_filter = len(df_news_raw)
    df_news_raw = df_news_raw[df_news_raw['week'] != 53]
    after_filter = len(df_news_raw)
    if before_filter != after_filter:
        print(f"Removed {before_filter - after_filter} week 53 entries (invalid weeks)")

    # Convert ISO week to Friday date to match freight data
    print("\nConverting ISO week numbers to Friday dates...")
    df_news_raw['date'] = df_news_raw.apply(iso_to_friday_date, axis=1)
    df_news_raw['date'] = pd.to_datetime(df_news_raw['date'])
    df_news_raw.set_index('date', inplace=True)
    print(f"Success: Converted dates. New range: {df_news_raw.index.min().date()} to {df_news_raw.index.max().date()}")

    # Drop the non-predictive global_worst_event_impact column
    if 'global_worst_event_impact' in df_news_raw.columns:
        df_news_raw = df_news_raw.drop(columns=['global_worst_event_impact'])
        print("Success: Dropped 'global_worst_event_impact' column.")

    # Save the cleaned data
    df_news_raw.to_csv('collected_news_data.csv')
    print(f"\nSuccess: New black swan data saved to 'collected_news_data.csv'")
    print(f"   Contains {len(df_news_raw.columns)} predictive features.")
    print("\nSample of new event data:")
    print(df_news_raw.head())

except FileNotFoundError:
    print(f"Error: The file '{news_file_path}' was not found.")
    print("Please ensure the BigQuery export CSV file is in the data folder.")
except Exception as e:
    print(f"An error occurred: {e}")

LOADING NEW BLACK SWAN GEOPOLITICAL DISRUPTION DATA
Success: Loaded 414 weekly records from data/bq-results-20251021-090045-1761037274833.csv
  Columns: ['iso_year', 'week', 'global_total_events', 'global_disruption_events', 'extreme_crisis_events', 'high_velocity_media_events', 'black_swan_candidate_events', 'global_avg_impact', 'global_worst_event_impact', 'global_avg_sentiment', 'global_total_media_mentions', 'global_peak_event_media', 'maritime_conflict_events', 'infrastructure_attack_events', 'trade_restriction_events', 'protest_events', 'middle_east_disruption', 'asia_disruption', 'europe_disruption', 'russia_ukraine_disruption', 'egypt_disruption', 'yemen_disruption', 'unique_sources']
Removed 7 week 53 entries (invalid weeks)

Converting ISO week numbers to Friday dates...
Success: Converted dates. New range: 2018-01-05 to 2025-10-24
Success: Dropped 'global_worst_event_impact' column.

Success: New black swan data saved to 'collected_news_data.csv'
   Contains 22 predictive fe

## Step 7: Save the collected data

We will save all three datasets as CSV files so we can use them in the next notebooks without having to fetch the data again.

In [8]:
# Save freight data
df_freight.to_csv('collected_freight_data.csv')
print("Saved freight data to 'collected_freight_data.csv'")
print(f"  {len(df_freight)} weekly records")

# Save oil data if we have it
if 'df_oil' in globals() and not df_oil.empty:
    df_oil.to_csv('collected_oil_data.csv')
    print("Saved oil price data to 'collected_oil_data.csv'")
    print(f"  {len(df_oil)} daily records")
else:
    print("No oil data to save (Yahoo Finance may be temporarily unavailable or df_oil not defined)")

print("\n" + "=" * 70)
print("DATA COLLECTION COMPLETE!")
print("=" * 70)
print("\nSummary of collected data:")
print(f"Available: Freight data: {len(df_freight)} weeks ({df_freight.index.min().strftime('%Y-%m-%d')} to {df_freight.index.max().strftime('%Y-%m-%d')})")
print(f"Available: Oil data: {len(df_oil) if 'df_oil' in globals() and not df_oil.empty else 0} days")

# Black swan disruption data may not exist if the file failed to load; guard against NameError
if 'df_news_raw' in globals():
    try:
        print(f"Available: Black swan disruption data: {len(df_news_raw)} weeks")
        print(f"  Features: {', '.join(df_news_raw.columns.tolist())}")
    except Exception:
        # If df_news_raw exists but isn't a dataframe or has issues, report its type
        print(f"df_news_raw exists but could not be summarized (type={type(df_news_raw)})")
else:
    print("Available: Black swan disruption data: 0 weeks (df_news_raw not found)")

print("\nReady for Step 2: Data Understanding and Feature Engineering")
print("=" * 70)

Saved freight data to 'collected_freight_data.csv'
  385 weekly records
Saved oil price data to 'collected_oil_data.csv'
  2787 daily records

DATA COLLECTION COMPLETE!

Summary of collected data:
Available: Freight data: 385 weeks (2018-01-05 to 2025-08-22)
Available: Oil data: 2787 days
Available: Black swan disruption data: 407 weeks
  Features: iso_year, week, global_total_events, global_disruption_events, extreme_crisis_events, high_velocity_media_events, black_swan_candidate_events, global_avg_impact, global_avg_sentiment, global_total_media_mentions, global_peak_event_media, maritime_conflict_events, infrastructure_attack_events, trade_restriction_events, protest_events, middle_east_disruption, asia_disruption, europe_disruption, russia_ukraine_disruption, egypt_disruption, yemen_disruption, unique_sources

Ready for Step 2: Data Understanding and Feature Engineering
